In [7]:
!pip install tflite-model-maker

这里安装libusb-1.0-0并更新apt，不然会找不到import

In [3]:
!sudo apt-get update

Hit:1 https://dl.yarnpkg.com/debian stable InRelease
Get:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3632 B]
Hit:3 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease    
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease       
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease    
Fetched 3632 B in 1s (3201 B/s)
Reading package lists... Done


In [4]:
!sudo apt-get install libusb-1.0-0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libusb-1.0-0 is already the newest version (2:1.0.23-2build1).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.


导入库：

In [1]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt



2024-05-31 08:53:11.914052: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-31 08:53:11.914088: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/codespace/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/codespace/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf

加载数据集

In [2]:
image_path = tf.keras.utils.get_file(
      'flower_photos.tgz',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')


228827136/228813984 [==============================] - 6s 0us/step


In [3]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)


2024-05-31 08:53:53.754812: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/codespace/.local/lib/python3.8/site-packages/cv2/../../lib64:
2024-05-31 08:53:53.754851: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-31 08:53:53.754879: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-6a58ee): /proc/driver/nvidia/version does not exist
2024-05-31 08:53:53.779982: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Load image with size: 3670, num_label: 5, labels: daisy, dandelion, roses, sunflowers, tulips.


训练模型

In [4]:
model = image_classifier.create(train_data)


INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


2024-05-31 08:54:11.779505: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-05-31 08:54:12.121000: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-05-31 08:54:12.238054: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-05-31 08:54:12.330496: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154140672 exceeds 10% of free system memory.
2024-05-31 08:54:12.390513: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 38535168 exceeds 10% of free system memory.


103/103 [==============================] - 90s 853ms/step - loss: 0.8662 - accuracy: 0.7640
Epoch 2/5
103/103 [==============================] - 85s 827ms/step - loss: 0.6514 - accuracy: 0.8905
Epoch 3/5
103/103 [==============================] - 87s 836ms/step - loss: 0.6225 - accuracy: 0.9111
Epoch 4/5
103/103 [==============================] - 84s 815ms/step - loss: 0.6017 - accuracy: 0.9260
Epoch 5/5
103/103 [==============================] - 83s 803ms/step - loss: 0.5871 - accuracy: 0.9311


评估模型

In [6]:
loss, accuracy = model.evaluate(test_data)


12/12 [==============================] - 12s 782ms/step - loss: 0.6579 - accuracy: 0.8692


保存模型

In [7]:
model.export(export_dir='.')


2024-05-31 09:04:22.108632: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpodg0d9oz/assets


INFO:tensorflow:Assets written to: /tmp/tmpodg0d9oz/assets
2024-05-31 09:04:26.504547: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-05-31 09:04:26.504709: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-05-31 09:04:26.537698: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 913 nodes (656), 923 edges (664), time = 15.02ms.
  function_optimizer: function_optimizer did nothing. time = 0.008ms.

/workspaces/Androidexam-21-041/.conda/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2024-05-31 09:04:27.202141: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Igno

INFO:tensorflow:Label file is inside the TFLite model with metadata.


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 3
INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpbuqah33k/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpbuqah33k/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite
